In [15]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

## User Input Arugment

In [17]:
#This input-argument used to: All data will be extracted since this date.
arg_date = '2021-08-07'

In [18]:
#Above arg is a string but we want a date-time obj. We also in reality need the data for the day before to calculate the 'change in % previous days price' column

arg_date_dt = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1) #Convert string into datetime obj and get previous date

In [19]:
arg_date_dt #One day before input-argument!

datetime.date(2021, 8, 6)

## Retrieving data from S3

In [20]:
s3 = boto3.resource('s3') #Have to do this always initally
bucket = s3.Bucket('deutsche-boerse-xetra-pds') #Retrieve the bucket with name from the server.

In [21]:
#Get all objects in S3 bucket starting from arg_date_dt
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], '%Y-%m-%d').date() >= arg_date_dt]

In [22]:
objects 

[s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-08-06/2021-08-06_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-08-06/2021-08-06_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-08-06/2021-08-06_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-08-06/2021-08-06_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-08-06/2021-08-06_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-08-06/2021-08-06_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-08-06/2021-08-06_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-08-06/2021-08-06_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-08-06/2021-08-06_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pd

In [8]:
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8') #Get an initial csv, just to set the columns of the df.
data = StringIO(csv_obj_init) #convert the CSV object into string format.
df_init = pd.read_csv(data, delimiter=',') #create a panda's df out of the csv

In [9]:
df_init

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades


In [10]:
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

In [11]:
df_all = pd.DataFrame(columns=df_init.columns) #giving structure of columns to facilitate appending to it later.

In [12]:
#More pythonic solution below
# for obj in objects:
#     csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8') #convert each filtered obj from bucket to a csv
#     data = StringIO(csv_obj) #convert the CSV object into string format into memory - so that it can be used without saving into hdd
#     df = pd.read_csv(data, delimiter=',') #create a panda's df out of the csv
#     df_all = df_all.append(df, ignore_index=True) #ignore_index: if there was any index, it will not be reset to a numerical one after append.

In [13]:
def csv_to_df(objname):
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8') #convert each filtered obj from bucket to a csv
    data = StringIO(csv_obj) #convert the CSV object into string format into memory - so that it can be used without saving into hdd
    df = pd.read_csv(data, delimiter=',') #create a panda's df out of the csv
    return df
    
df_all = df_all.append([csv_to_df(obj.key) for obj in object], ignore_index=True) #ignore_index: if there was any index, it will not be reset to a numerical one after append.

TypeError: 'type' object is not iterable

In [ ]:
df_all

# Selecting the specific rows that we need for our wanted report

In [ ]:

columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
#its easy to filter:
df_all = df_all.loc[:, columns] #all rows but we want only specific columns

In [ ]:
df_all

In [ ]:
df_all.dropna(inplace=True) #drop any empty row (just to make sure - even though unlikely to be any)
df_all

# Doing the transformations to get the report

## Get opening price per ISIN and Day

In [ ]:
df_all[['ISIN', 'Date', 'StartPrice']]

In [ ]:
#creating new column
#transform('first') is calling a group-by method. It returns the first non-NaN value in a series, or NaN if there is none on each row of the 'StartPrice' columns and through '.transform' it returns this columns - to be set as a new column.
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('first') #doesnt appear to have achieved anything
#transform('first'): It returns a series / dataframe with the same shape as the source group chunk, in which all values in every individual column are replaced with the first non-NaN value in this column, or with NaN if there is none.

df_all

In [ ]:
df_all[df_all['ISIN']=='AT0000A0E9W5']

## Get closing price per ISIN and Day

In [ ]:
df_all['closing_price'] = df_all.sort_values(by='Time').groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

df_all

In [ ]:
df_all[df_all['ISIN']=='DE000A0TGJ55']

## Aggregations to create most of the desired columns

In [ ]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=("TradedVolume", 'sum'))

In [ ]:
df_all

## Get column that shows the percentage of change in previous days closing price to todays closing price.

In [ ]:
df = pd.DataFrame({"Col1": [10, 20, 15, 30, 45],
                   "Col2": [13, 23, 18, 33, 48],
                   "Col3": [17, 27, 22, 37, 52]},
                  index=pd.date_range("2020-01-01", "2020-01-05"))

df

In [ ]:
df.shift(1)

In [ ]:
df_all['closing_price_eur'] #ignore

In [ ]:
df_all.sort_values(by=['Date']).groupby(['ISIN']).shift(1)['closing_price_eur'] #ignore

In [ ]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1) 

In [ ]:
df_all

In [ ]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

In [ ]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [ ]:
df_all = df_all.round(decimals=2)

In [ ]:
df_all.Date

In [ ]:
#Filtering by the passed-in-argument data, as the report should not show older data.
df_all = df_all[df_all.Date >= arg_date]

In [ ]:
df_all

## Write to S3

In [ ]:
#creating the key (name) of the object
key = 'stocks_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet' #will store it as a parquet file as it is better practise and more efficiently stored.

In [ ]:
out_buffer = BytesIO() #to handle binary data in memory
df_all.to_parquet(path=out_buffer, index=False) #Write a DataFrame as a parquet format (a file format) into BytesIO (memory)

bucket_target = s3.Bucket("stocks-etl-project-essa") #Creating bucket instance for the target bucket (where we will store)
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key) #uploading the file

## Reading the uploaded file

In [ ]:
for obj in bucket_target.objects.all():
    print(obj.key) #checking if the file exists in s3

In [ ]:
prq_obj = bucket_target.Object(key='stocks_daily_report_20210821_031514.parquet').get().get('Body').read()
data = BytesIO(prq_obj) #pandas only accepts a file on disk or a file-like-object (BytesIO is a file-like-obj)
df_report = pd.read_parquet(data) #using pandas built in function

In [14]:
df_report

NameError: name 'df_report' is not defined